In [1]:
import pandas as pd
import glob
from datetime import datetime, timedelta



In [2]:
Path = r'E:\_Projects\Schedule Stability\\'
Completion_Path = r'E:\_Projects\completions\reports\Completion.xlsx'

In [3]:

#################### User Input
Date = input("Enter Monday date in the format (YYYY-MM-DD) :")
Weeks = input('Enter Week : ')
Year = Date[0:4]


B_V = 0
C_V = 0


try:
    Bud_file = glob.glob(Path+'Input\\Weekly\\Open_Orders_Atleos_??????.xlsx')
    print('BUD Plan File : ',Bud_file)
    for Bud_file in Bud_file:
        BUD = pd.read_excel(Bud_file,sheet_name='Data_Source')
    B_V = 1
except:
    print('BUD File Not Found')
    BUD = pd.DataFrame()

try:
    Che_file = glob.glob(Path+'Input\\Weekly\\Wk??.xlsx')
    print('Chennai Plan File : ',Che_file)
    for Che_file in Che_file:
        CHE_ATM = pd.read_excel(Che_file)

        
    C_V = 1    
except:
    print('Chennai File Not Found')
    CHE_ATM = pd.DataFrame()

    
try: 
    Glob_Doc = glob.glob(Path+'Input\\Weekly\\Global DOC Atleos?????????.xlsb')
    print('Global Doc File : ',Glob_Doc)
    for Glob_Doc in Glob_Doc:
        GLOB_DOC =pd.read_excel(Glob_Doc,sheet_name='Orders',engine='pyxlsb')
except:
    print('Global file Not found')
    

Product_Range =pd.read_excel(Path+'Product Range.xlsx')

    
print('Chennai : ',C_V)
print('BUD : ', B_V )


COMP = pd.read_excel(Completion_Path,sheet_name='Data')

COMP['ship_date'] = pd.to_datetime(COMP['ship_date']).dt.date

####################### Start Date and End Date
Start_Date = pd.to_datetime(Date).date()
End_Date = Start_Date + timedelta(days=6)

CHE_St_Dt = Start_Date - timedelta(days=1)
CH_Ed_Dt = CHE_St_Dt + timedelta(days=6)

Enter Monday date in the format (YYYY-MM-DD) : 2025-09-01
Enter Week :  36


BUD Plan File :  ['C:\\Vasanthan\\Report\\Schedule Stability\\Schedule Stability Weekly\\Automation\\\\Input\\Open_Orders_Atleos_01_Sep.xlsx']
Chennai Plan File :  ['C:\\Vasanthan\\Report\\Schedule Stability\\Schedule Stability Weekly\\Automation\\\\Input\\Wk04.xlsx']
Global Doc File :  ['C:\\Vasanthan\\Report\\Schedule Stability\\Schedule Stability Weekly\\Automation\\\\Input\\Global DOC Atleos 20250901.xlsb']
Chennai :  1
BUD :  1


In [5]:
print('Start Date :',Start_Date)
print('End Date : ' ,End_Date)
print('Chennai Start Date : ', CHE_St_Dt)
print('Chennai End Date : ', CH_Ed_Dt)

Start Date : 2025-09-01
End Date :  2025-09-07
Chennai Start Date :  2025-08-31
Chennai End Date :  2025-09-06


### ################ Plan Start ##############

In [6]:

if B_V == 1:
    BUD['Update SSD'] = pd.to_datetime(BUD['Update SSD']).dt.date
    
    
    BUD = BUD.loc[(BUD['Update SSD'] >= Start_Date) & (BUD['Update SSD'] <= End_Date) & (BUD['Material'].str.contains('MC'))]
    
    BUD['Plant'] = 'BUD'
    
    BUD = BUD[BUD['Product Type']=='Atleos']
    
    BUD = BUD[['Material','Open Qty','Plant']]
    
    
    BUD.rename(columns={'Material': 'Product ID'},inplace=True)
    BUD.rename(columns={'Open Qty': 'Plan Qty'},inplace=True)
else:
    print("BUD is Skipping")

################

if C_V==1:
    CHE = pd.concat([CHE_ATM],axis=0)
    
    CHE = CHE[['MCID','Qty']]
    CHE = CHE.loc[(CHE['MCID'].str.contains('MC'))]
    CHE['Plant'] = 'CHE'
    
    
    CHE.rename(columns={'MCID':'Product ID'},inplace=True)
    CHE.rename(columns={'Qty':'Plan Qty'},inplace=True)
else:
    print("CHE is Skipping")
    
    
########### Concat
Plan = pd.DataFrame()

if ((C_V==1) & (B_V==1)):
    Plan = pd.concat([CHE,BUD],axis=0)
    
elif((C_V==1) & (B_V==0)):
    Plan = CHE

elif((C_V==0) & (B_V==1)):
    Plan = BUD
    
else:
    print("No plant data")



Plan = Plan.groupby(['Plant','Product ID']).agg({'Plan Qty':'sum'})


Plan.to_csv(Path+'\\Output\\ref\\Plan.csv')


C:\Users\vv185114\AppData\Local\Temp\ipykernel_19700\607082568.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  BUD['Plant'] = 'BUD'


### ################################### PLAN COMPLETED

### ######################## Completion

In [8]:

Other_Comp = pd.DataFrame()

if((B_V==1)):
    Other_Comp = COMP.loc[((COMP['ship_date']>=Start_Date) & (COMP['ship_date']<=End_Date)) & (COMP['item'].str.contains('MC')) & (COMP['site'].astype(str).isin(['BUD'])) & (COMP['prod_grp_wb'].str.contains('ATM'))]

    Other_Comp = Other_Comp[['site','item','prod_grp_wb','qty']]
else:
    print("*********** Completion Not found in ENN ***********")
    

if C_V==1:
    CHE_Comp = COMP.loc[((COMP['ship_date']>=CHE_St_Dt) & (COMP['ship_date']<=CH_Ed_Dt)) & (COMP['item'].str.contains('MC')) & (COMP['site'].astype(str).isin(['CHS','CHE'])) & (COMP['prod_grp_wb'].astype(str).isin(['ATM']))]
    
    CHE_Comp['site'] = CHE_Comp['site'].replace('CHS','CHE')
    
    CHE_Comp = CHE_Comp[['site','item','prod_grp_wb','qty']]
else:
    print("*********** Completion Not found in Chennai ***********")

COMP = pd.DataFrame()


if((C_V==1) & (B_V==1)):
    COMP = pd.concat([Other_Comp,CHE_Comp],axis=0)
    
 
elif((C_V==1) &  (B_V==0)):
    COMP = CHE_Comp

elif((C_V==0) &  (B_V==1)):
    Plan = Other_Comp
    
else:
    print("No plant data")


COMP = COMP.groupby(['site','item','prod_grp_wb']).agg({'qty':'sum'})

COMP.to_csv(Path+'\\Output\\ref\\Completion.csv')


C:\Users\vv185114\AppData\Local\Temp\ipykernel_19700\3090349217.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CHE_Comp['site'] = CHE_Comp['site'].replace('CHS','CHE')


### ################################# Plan Vs Completion ###########################

In [9]:
Plan = pd.read_csv(Path+'\\Output\\ref\\Plan.csv')
COMP = pd.read_csv(Path+'\\Output\\ref\\Completion.csv')


Plan['Concat'] = Plan['Plant']+Plan['Product ID']
COMP['Concat'] = COMP['site']+COMP['item']




Plan = pd.merge(Plan, COMP,on='Concat',how='outer')

Plan = Plan.rename(columns=lambda x: x.replace('qty', 'Actual Completions'))

Plan.loc[Plan['Plant'].isnull(),'Plant'] = Plan['site']
Plan.loc[Plan['Product ID'].isnull(),'Product ID'] = Plan['item']

Plan['Actual Completions'] = Plan['Actual Completions'].fillna(0)
Plan['Plan Qty'] = Plan['Plan Qty'].fillna(0)
Plan = Plan[['Plant','Product ID','Plan Qty','Actual Completions']]

Plan['Concat'] = Plan['Plant']+Plan['Product ID']


Plan.loc[(Plan['Plan Qty']-Plan['Actual Completions']<=0),'Planned Completions']=Plan['Plan Qty']
Plan.loc[(Plan['Plan Qty']-Plan['Actual Completions']>0),'Planned Completions']=Plan['Actual Completions']

Plan.loc[(Plan['Plan Qty']-Plan['Actual Completions'])==0,'Result']='CLEAN'
Plan.loc[(Plan['Plan Qty']-Plan['Actual Completions'])<0,'Result']='UNPLANNED'
Plan.loc[(Plan['Plan Qty']-Plan['Actual Completions'])>0,'Result']='MISS'

Plan.loc[(Plan['Plan Qty']-Plan['Actual Completions'])>0,'Miss Qty']=(Plan['Plan Qty']-Plan['Actual Completions'])
Plan.loc[(Plan['Plan Qty']-Plan['Actual Completions'])<0,'Unplanned Qty']=(Plan['Actual Completions'] - Plan['Plan Qty'])


Plan.loc[(Plan['Plan Qty']-Plan['Actual Completions'])<=0,'Clean Qty']=Plan['Plan Qty']

Plan['Class'] = Plan['Product ID'].astype(str).str[:4]



Plan['Class'] = Plan['Class'].astype(str)
Product_Range['Class'] = Product_Range['Class'].astype(str)


Plan = pd.merge(Plan, Product_Range,on='Class',how='left')



### ######################## UPDATING CUSTOMER NAME ###############################

In [10]:
GLOB_DOC = GLOB_DOC[['PID','Whse','Customer/End Customer']]

GLOB_DOC = GLOB_DOC[(GLOB_DOC['Customer/End Customer']!='NCR INTERCOMPANY') & GLOB_DOC['Customer/End Customer'].notna()]


GLOB_DOC['Whse'] = GLOB_DOC['Whse'].replace('ENN','BUD')


GLOB_DOC['Concat'] = GLOB_DOC['Whse']+GLOB_DOC['PID']


GLOB_DOC.to_csv(Path+'\\Output\\ref\\GLOB_DOC.csv')


GLOB_DOC = GLOB_DOC.drop_duplicates(subset=['PID','Whse'], keep='first')

Plan = pd.merge(Plan, GLOB_DOC,on='Concat',how='left')

Plan = Plan.rename(columns=lambda x: x.replace('Customer/End Customer', 'Customer'))

Plan['Product ID2']=''

Plan['Weeks'] = Weeks
Plan['Year'] = Year

Plan = Plan[['Plant','Weeks','Year','Product Range','Prod Range 2','Class','Product ID2','Product ID','Customer','Plan Qty','Actual Completions','Planned Completions','Result','Miss Qty','Unplanned Qty','Clean Qty']]

Plan['Customer'] = Plan['Customer'].fillna('Other-')

Plan.to_csv(Path+'\\Output\\Schedule Stability Weekly.csv',index=False)



In [11]:

Miss = Plan[(Plan['Result']=='MISS')]

Miss = Miss[['Plant','Product Range','Product ID','Customer','Miss Qty','Result']]

Miss.to_csv(Path+'\\Output\\Miss.csv',index=False)



In [12]:
#############################################

BUD_Plan = Plan[Plan['Plant']=='BUD']
CHE_Plan_ATM = Plan[(Plan['Plant']=='CHE')&(Plan['Product Range']=='ATM')]


BUD_Plan_Qty = BUD_Plan['Plan Qty'].sum()
BUD_Actual_Completions = BUD_Plan['Actual Completions'].sum()

CHE_Plan_ATM_Qty = CHE_Plan_ATM['Plan Qty'].sum()
CHE_Actual_ATM_Completions = CHE_Plan_ATM['Actual Completions'].sum()



print('**********************************************')
print('BUD Plan       : ', BUD_Plan_Qty)
print('CHE Plan ATM   : ', CHE_Plan_ATM_Qty)


print('**********************************************')
print('BUD Completion     : ',BUD_Actual_Completions)
print('CHE ATM Completion : ',CHE_Actual_ATM_Completions)


print('**REPOTE AUTOMATION COMPLETED**')

**********************************************
BUD Plan       :  395.0
CHE Plan ATM   :  1182.0
**********************************************
BUD Completion     :  435.0
CHE ATM Completion :  1475.0
**REPOTE AUTOMATION COMPLETED**
